## Imports

In [2]:
import os
import numpy as np
import pandas as pd
from unidecode import unidecode
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec
import seaborn as sns
from google.cloud import storage
from ydata_profiling import ProfileReport
import math
import requests
import bs4
import re

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
pd.set_option('display.max_columns', 100)

/home/arthur/anaconda3/envs/busca-apartamentos-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Funções auxiliares

In [3]:
def get_data(bucket_name:str, imobiliarias:str = ['apolar', 'cilar'],by:str = ['date','date_diff'], dates:list = [], date_diff:int = 2):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    files_on_bucket = [i.name for i in bucket.list_blobs()]
    files = pd.DataFrame(files_on_bucket, columns=['name'])
    files['date'] = pd.to_datetime(files['name'].apply(lambda f: f.split(' - ')[0]))
    files['imobiliaria'] = files['name'].apply(lambda f: f.split(' - ')[-1].replace('.csv',''))

    match by:
        case 'date':
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
        case 'date_diff':
            dates = files['date'].sort_values(ascending=False).drop_duplicates().reset_index(drop=True)[:date_diff].tolist()
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
    
    df_full = pd.DataFrame()

    for file_name in files['name'].tolist():
        try:
            df_aux = pd.read_csv(f'gs://{bucket_name}/{file_name}')
            df_full = pd.concat([df_full, df_aux], axis = 0)
        except:
            pass

    df_full = df_full.reset_index(drop=True)

    return df_full

def barplot(title:str, 
            group:str, 
            agg:str, 
            agg_name:str, 
            data:pd.DataFrame, 
            agg_func:str, 
            figure= plt.figure, 
            title_font_size:int =10, 
            figsize=(10,5),
            subplot:plt.subplot = None, 
            grid:list = None, 
            orient:str='h',
            label=True,
            rotation_label:int = 45,
            position_label:str = 'center',
            color_label:str = 'white',
            size_label:str = 'small',
            fmt:str = '%.0f',
            sort: bool = True, 
            hue:str = None,
            stacked:bool = False):
    
    group_list = [group]
    if hue:
        group_list.append(hue)

    # group data
    aux = data[group_list + [agg]].groupby(group_list).agg(agg_func).reset_index().rename(columns={agg:agg_name})

    if sort:
        aux = aux.sort_values(agg_name, ascending=False)
        
    # plot
    if subplot:
        subplot(grid)
    else:
        figure(figsize=figsize)

    # plot configs
    plt.title(title, fontsize=title_font_size)
    plt.xticks(rotation = rotation_label)

    # figure
    if orient == 'h':
        g = sns.barplot(x = group, y = agg_name, hue = hue, dodge = not stacked, data = aux)
    elif orient == 'v':
        g = sns.barplot(y = group, x = agg_name, hue = hue, dodge = not stacked, data = aux)
    else:
        raise("Variável 'orient' informada não é válida")

    if label:
        for i in g.containers:
            g.bar_label(i, color = color_label, label_type=position_label, fontsize = size_label, fmt = fmt)
    else:
        pass

def histplot(title:str, col:str, data:pd.DataFrame, figsize=(10,5), label=True):

    plt.figure(figsize=figsize)
    plt.title(title)
    g = sns.histplot(x = col,  data = data)
    plt.xticks(rotation = 45)
    if label:
        for i in g.containers:
            g.bar_label(i, color = 'white',label_type='center')
    else:
        pass

def get_infos_curitiba():

    ## Request site
    response = requests.get('https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Curitiba')

    ## beautiful soup object
    soup = bs4.BeautifulSoup(response.content, 'html.parser')

    # tabelas da wikipedia
    infos_tabela = soup.findAll('table', {'class','wikitable'})

    bairros_info_list = []

    ## para cada tabela
    for tabela in infos_tabela:

        # colunas
        columns = [i.text.replace('\n','') for i in tabela.findAll('th')]

        # valores da tabela
        regiao = ' '.join([i.text.replace('\xa0','').replace('\n','') for i in tabela.findAll('td')][0].replace('Bairros oficiais de Curitiba - Regional ','').split(' ')[:-1])
        table_values = [i.text.replace('\xa0','').replace('\n','') for i in tabela.findAll('td')][1::]
        table_values_list = [] 
        for i in range(0,len(table_values),7): 
            table_values_list.append(table_values[i:i+7])

        # preenchenco dicionário
        for b in table_values_list:

            bairros_info_dict = {}
            bairros_info_dict['Região'] = regiao
            bairros_info_dict['Bairro'] = b[0]
            bairros_info_dict['Área (km²)'] = b[1].replace(',','.')
            bairros_info_dict['Homens'] = b[2].replace(',','.')
            bairros_info_dict['Mulheres'] = b[3].replace(',','.')
            bairros_info_dict['Total'] = b[4].replace(',','.')
            bairros_info_dict['Domicilios particulares'] = b[5].replace(',','.')
            bairros_info_dict['Rendimento mensal médio porresponsáveis dos domicílios (R$)'] = b[6].replace(',','.')

            bairros_info_list.append(bairros_info_dict)
    
    return pd.DataFrame(bairros_info_list)

def busca_e_preenche_nulos(data, column, string_search):

    if isinstance(string_search, list):
        for s in string_search:
            data.loc[data[column] == 0, column] = np.nan
            data.loc[data[column].isna(), column] = data.loc[data[column].isna(), 'descricao'].apply(lambda x: 0 if isinstance(x,float) else 
                                                                                                            1 if s in unidecode(x.lower()) else 0)
    elif isinstance(string_search,str):
        data.loc[data[column] == 0, column] = np.nan
        data.loc[data[column].isna(), column] = data.loc[data[column].isna(), 'descricao'].apply(lambda x: 0 if isinstance(x,float) else 
                                                                                                            1 if string_search in unidecode(x.lower()) else 0)
    else:
        raise("Atributo 'string_search' deve ser uma lista ou uma string" )
    
    return data

def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

def vagas_garagem(descricao):
    # Procura por padrões do tipo 'Garagem: [quantidade]' na descrição
    padrao = r'Garagem: (\d+)'
    resultado = re.search(padrao, descricao)

    if resultado:
        # Se encontrou, retorna a quantidade de vagas
        return int(resultado.group(1))
    else:
        # Se não encontrou ou não há informação sobre vagas, retorna 0
        return 0

def histplot_matrix(data, figsize:tuple = (10,5)):
    n_features = data.shape[1]

    rows = int(np.floor(np.sqrt(n_features)))
    cols = int(np.ceil(n_features/rows))

    grid = gridspec.GridSpec(rows,cols)

    for row in range(rows):
        for col in range(cols):
            n = (row*cols) + col
            if n >= n_features:
                break
            plt.subplot(grid[row,col])
            column_name = data.columns.tolist()[n]
            plt.title(column_name)
            data[column_name].hist(figsize=figsize)


    plt.tight_layout()

# Carregando Dados

In [4]:
bucket_name='busca-apartamentos-bucket'
imobiliaria = 'apolar'

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

datas = set([i.name.split(' - ')[0] for i in bucket.list_blobs() if i.name.split(' - ')[-1] == f'{imobiliaria}.csv'])

data = get_data(bucket_name='busca-apartamentos-bucket',by='date', dates=datas, imobiliarias=['apolar'])

In [5]:
df = data.copy()

# Análise Descritiva dos Dados

In [8]:
df['site'].value_counts()

site
Apolar    20902
Name: count, dtype: int64

In [9]:
df['data_coleta'].value_counts()

data_coleta
2024-03-06    668
2024-03-07    653
2024-03-02    652
2024-02-28    632
2024-03-17    386
2024-03-31    382
2024-02-21    379
2024-03-14    379
2024-02-24    378
2024-05-25    378
2024-04-27    378
2024-03-19    377
2024-03-26    377
2024-05-18    375
2024-02-22    375
2024-03-24    375
2024-05-14    373
2024-03-10    373
2024-03-09    373
2024-04-13    373
2024-06-08    372
2024-05-11    372
2024-07-27    372
2024-06-11    372
2024-06-22    372
2024-07-20    372
2024-06-15    372
2024-05-21    371
2024-08-06    369
2024-07-23    369
2024-08-10    369
2024-04-06    368
2024-04-16    368
2024-05-07    367
2024-03-12    367
2024-04-20    367
2024-04-09    363
2024-04-23    361
2024-07-06    360
2024-06-18    360
2024-06-25    360
2024-07-13    360
2024-07-16    358
2024-05-04    358
2024-06-04    358
2024-06-29    357
2024-07-09    348
2024-06-01    348
2024-08-03    348
2024-02-27    348
2024-04-02    345
2024-04-30    343
2024-07-30    336
2024-07-02    336
Name: count, dty

In [10]:
df['titulo'].value_counts()

titulo
\nref.                                                                                                          835
\n                Apartamento Residencial para Locação no Água Verde\n                ref. 102991                46
\n                Apartamento Residencial para Locação no Capão Raso\n                ref. 87678                 46
\n                Apartamento Residencial para Locação no Centro\n                ref. 104000                    45
\n                Apartamento Residencial para Locação no Bairro Alto\n                ref. 106303               45
                                                                                                               ... 
\n                Apartamento Residencial para Locação no Cidade Industrial\n                ref. 106069          1
\n                Apartamento Residencial para Locação no Ecoville\n                ref. 20793                    1
\n                Apartamento Residencial para Locação no Bigorri

In [11]:
df.groupby('link').agg(n_anuncios=('link','count')).reset_index().groupby('n_anuncios').agg(total=('link','count')).sort_values('total',ascending=False)

,total
n_anuncios,
4,289
3,99
11,97
13,90
15,84
12,69
16,68
2,64
17,63


In [12]:
df.groupby('endereco').agg(n_anuncios=('endereco','count')).reset_index().groupby('n_anuncios').agg(total=('endereco','count')).sort_values('total',ascending=False)

,total
n_anuncios,
2,327
4,154
1,77
11,59
13,54
...,...
57,1
54,1
52,1


In [13]:
df['valores']

0        Aluguel R$ 20.000,00 Seguro Incêndio R$ 62,56T...
1        Aluguel R$ 12.000,00 Condomínio R$ 840,00 Segu...
2        Aluguel R$ 10.500,00 Condomínio R$ 1.549,67 Se...
3        Aluguel R$ 7.500,00 Condomínio R$ 1.690,58 Seg...
4        Aluguel R$ 5.000,00 Condomínio R$ 635,61 Segur...
                               ...                        
20897    R$ 700,00, , Condomínio R$ 446,84, IPTU R$ 20,...
20898    R$ 700,00, , Condomínio R$ 340,00, IPTU R$ 22,...
20899    R$ 680,00, , Condomínio R$ 360,00, IPTU R$ 20,...
20900    R$ 600,00, , Condomínio R$ 451,09, Seguro Incê...
20901    R$ 600,00, , Condomínio R$ 362,00, IPTU R$ 40,...
Name: valores, Length: 20902, dtype: object

In [14]:
def busca_substring(substring, string_list):
    result = np.nan
    for s in string_list:
        if substring in s:
            try:
                result = re.findall(r'\s(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)', s)[0]
            except:
                result = s
            break
            
    return result

def separa_valores_imovel(string):

    # Padrao regex para encontrar nome e valor monetário
    padrao = r'(\w+)\sR\$\s(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)'

    # Encontrar todas as correspondências na string
    correspondencias = re.findall(padrao, string)

    # Imprimir os resultados
    list_values = []
    for correspondencia in correspondencias:
        nome, valor = correspondencia
        list_values.append(f'{nome}: {valor}')
    
    return list_values

def formata_valores(valores):
    return valores.str.strip().str.replace('.','').apply(lambda x: x if pd.isna(x) else x.split(',')[0]).astype('float64')

In [15]:
df['endereco'].apply(lambda x: x if pd.isna(x) else x.replace('\n','').strip().split(', ')[-1])

0              Água Verde - Curitiba
1              Cristo Rei - Curitiba
2                Ecoville - Curitiba
3                  Mercês - Curitiba
4             Guabirotuba - Curitiba
                    ...             
20897           Ganchinho - Curitiba
20898           Cachoeira - Curitiba
20899    Campo De Santana - Curitiba
20900      Campo Comprido - Curitiba
20901           Tatuquara - Curitiba
Name: endereco, Length: 20902, dtype: object

In [16]:
df['titulo'] = df['titulo'].apply(lambda x: x if pd.isna(x) else x.replace('\n','').strip())
df['endereco'] = df['endereco'].apply(lambda x: x if pd.isna(x) else x.replace('\n','').strip())
df['descricao'] = df['descricao'].apply(lambda x: x if pd.isna(x) else x.replace('\n','').strip())

df['bairro'] = df['endereco'].str.strip().apply(lambda x: x if pd.isna(x) else unidecode(x.replace('\n','').strip().split(', ')[-1].split(' - ')[0].capitalize()))

# Valores
df['aluguel'] = df['valores'].apply(lambda x: x if pd.isna(x) else 
                    x.split(', ,')[0].replace('R$ ','').strip() if "Aluguel" not in x else
                    x.split(', ,')[0].replace('R$ ','').replace('Aluguel ','').strip() if "Aluguel" in x else
                    x)
df['aluguel'] = df['aluguel'].apply(lambda x: x if pd.isna(x) else x.split(' ')[0])
df['condominio'] = df['valores'].apply(lambda x: busca_substring('Condomínio', separa_valores_imovel(x)) if not pd.isna(x) else x)
df['iptu'] = df['valores'].apply(lambda x: busca_substring('IPTU', separa_valores_imovel(x)) if not pd.isna(x) else x)
df['seguro_incendio'] = df['valores'].apply(lambda x: busca_substring('Incêndio', separa_valores_imovel(x)) if not pd.isna(x) else x)

df['aluguel'] = formata_valores(df['aluguel'])
df['condominio'] = formata_valores(df['condominio'])
df['iptu'] = formata_valores(df['iptu'])
df['seguro_incendio'] = formata_valores(df['seguro_incendio'])

# Atributos
df['area'] = df['atributos'].apply(lambda x: x if pd.isna(x)  else busca_substring('m²', x.split(', '))).str.replace('m²','')
df['banheiros'] = df['atributos'].apply(lambda x: x if pd.isna(x)  else busca_substring('banheiro', x.split(', '))).str.replace('banheiro','')
df['quartos'] = df['atributos'].apply(lambda x: x if pd.isna(x)  else busca_substring('quarto', x.split(', '))).str.replace('quartos','')
df['suites'] = df['atributos'].apply(lambda x: x if pd.isna(x)  else busca_substring('suite', x.split(', '))).str.replace('suite','')
df['vagas_garagem'] = df['atributos'].apply(lambda x: x if pd.isna(x) else busca_substring('vagas', x.split(', '))).str.replace('vagas','')

# Detalhes do imóvel/condomínio
df['mobiliado'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festas' in unidecode(x.lower()) else 'Não')

In [17]:
# site
# data_coleta
# titulo
# link
# aluguel
# condominio
# iptu
# seguro_incendio
# area
# banheiros
# quartos
# suites
# vagas_garagem
# descricao
# ficha tecnica

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica,bairro,aluguel,condominio,iptu,seguro_incendio,area,banheiros,quartos,suites,vagas_garagem,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
0,Apolar,2024-02-21,Apartamento Residencial para Locação no Água V...,https://www.apolar.com.br/alugar/curitiba/agua...,"Avenida Silva Jardim, 2733, Água Verde - Curitiba","Aluguel R$ 20.000,00 Seguro Incêndio R$ 62,56T...",203 m²,Apartamento com ...,NaN,Agua verde,20000.0,NaN,NaN,62.0,203,NaN,NaN,NaN,NaN,Não,Não,Não,Não,Não,Não
1,Apolar,2024-02-21,Apartamento Residencial para Locação no Cristo...,https://www.apolar.com.br/alugar/curitiba/cris...,"Rua Urbano Lopes, 238, Cristo Rei - Curitiba","Aluguel R$ 12.000,00 Condomínio R$ 840,00 Segu...","250 m², 1 banheiro, 3 vagas, 3 quartos, 1 suit...",Apartamento com 3 qua...,NaN,Cristo rei,12000.0,840.0,NaN,71.0,250,1,3,1,3,Sim,Não,Sim,Sim,Sim,Sim
2,Apolar,2024-02-21,Apartamento Residencial para Locação no Ecovil...,https://www.apolar.com.br/alugar/curitiba/ecov...,"Rua Dep. Heitor Alencar Furtado, 3180, Ecovill...","Aluguel R$ 10.500,00 Condomínio R$ 1.549,67 Se...","330 m², 2 vagas, 4 quartos",Apartamento com 4 qua...,NaN,Ecoville,10500.0,1549.0,NaN,87.0,330,NaN,4,NaN,2,Não,Não,Não,Não,Não,Não
3,Apolar,2024-02-21,Apartamento Residencial para Locação no Mercês...,https://www.apolar.com.br/alugar/curitiba/merc...,"Avenida Cândido Hartmann, 330, Mercês - Curitiba","Aluguel R$ 7.500,00 Condomínio R$ 1.690,58 Seg...","163 m², 2 banheiros, 2 vagas, 2 quartos, 1 sui...",Apartamento com 2 qua...,NaN,Merces,7500.0,1690.0,NaN,53.0,163,2 s,2,1,2,Não,Não,Sim,Não,Sim,Não
4,Apolar,2024-02-21,Apartamento Residencial para Locação no Guabir...,https://www.apolar.com.br/alugar/curitiba/guab...,"Avenida Senador Salgado Filho, 1798, Guabirotu...","Aluguel R$ 5.000,00 Condomínio R$ 635,61 Segur...","107 m², 4 banheiros, 2 vagas, 2 quartos",Apartamento com 2 qua...,NaN,Guabirotuba,5000.0,635.0,NaN,39.0,107,4 s,2,NaN,2,Não,Não,Não,Sim,Sim,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20897,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/ganc...,"Rua Alexandro Glenski, 815, Ganchinho - Curitiba","R$ 700,00, , Condomínio R$ 446,84, IPTU R$ 20,...","46 m², 2 banheiros, 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 46.39m², Área ...",Ganchinho,700.0,446.0,20.0,26.0,46,2 s,2,NaN,NaN,Não,Não,Não,Não,Não,Sim
20898,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/cach...,"Rua Howell Lewis Fry, 341, Cachoeira - Curitiba","R$ 700,00, , Condomínio R$ 340,00, IPTU R$ 22,...","52 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 52m², Área Tot...",Cachoeira,700.0,340.0,22.0,26.0,52,1,2,NaN,NaN,Não,Não,Não,Não,Não,Sim
20899,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Ângelo Tozim, 1551, Campo De Santana - Cur...","R$ 680,00, , Condomínio R$ 360,00, IPTU R$ 20,...","45 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 45m², Área Tot...",Campo de santana,680.0,360.0,20.0,26.0,45,1,2,NaN,NaN,Não,Não,Não,Sim,Não,Não
20900,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Íris Antônio Campos, 250, Campo Comprido -...","R$ 600,00, , Condomínio R$ 451,09, Seguro Incê...","40 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 40m², Área Tot...",Campo comprido,600.0,451.0,NaN,26.0,40,1,2,NaN,NaN,Sim,Não,Não,Não,Não,Não


In [30]:
df.iloc[10416]

site                                                          Apolar
data_coleta                                               2024-04-30
titulo                                                          ref.
link               https://www.apolar.com.br/alugar/curitiba/alto...
endereco                                                      , ,  -
valores                                                     R$ 0,00,
atributos                                                        NaN
descricao                                                        NaN
ficha_tecnica                                                    NaN
bairro                                                             -
aluguel                                                          0.0
condominio                                                       NaN
iptu                                                             NaN
seguro_incendio                                                  NaN
area                              

In [52]:
df['endereco'].unique().tolist()

['Avenida Silva Jardim, 2733, Água Verde - Curitiba',
 'Rua Urbano Lopes, 238, Cristo Rei - Curitiba',
 'Rua Dep. Heitor Alencar Furtado, 3180, Ecoville - Curitiba',
 'Avenida Cândido Hartmann, 330, Mercês - Curitiba',
 'Avenida Senador Salgado Filho, 1798, Guabirotuba - Curitiba',
 'Rua Schiller, 126, Cristo Rei - Curitiba',
 'Rua Coronel Joaquim Ignácio Taborda Ribas, 864, Bigorrilho - Curitiba',
 'Alameda Júlia Da Costa, 2242, Bigorrilho - Curitiba',
 'Rua Alberto Bolliger, 345, Juvevê - Curitiba',
 'Avenida República Argentina, 2751, Portão - Curitiba',
 'Rua Martim Afonso, 1918, Bigorrilho - Curitiba',
 'Rua Riachuelo, 110, Centro - Curitiba',
 'Avenida Presidente Getúlio Vargas, 881, Rebouças - Curitiba',
 'Rua Doutor Alexandre Gutierrez, 268, Água Verde - Curitiba',
 'Rua Euclides Bandeira, 500, Centro Cívico - Curitiba',
 'Rua Amazonas, 691, Água Verde - Curitiba',
 'Avenida Anita Garibaldi, 336, Cabral - Curitiba',
 'Rua Vinte E Quatro De Maio, 980, Rebouças - Curitiba',
 ', ,

In [79]:
aux = df.groupby('link').agg(data_inicio=('data_coleta', 'min'), data_fim=('data_coleta','max')).reset_index()

aux['tempo'] = (pd.to_datetime(aux['data_fim']) - pd.to_datetime(aux['data_inicio']))

aux = aux.loc[aux['tempo'] == '8 days']

aux

,link,data_inicio,data_fim,tempo
0,https://www.apolar.com.br/alugar/almirante-tam...,2024-02-28,2024-03-07,8 days
1,https://www.apolar.com.br/alugar/almirante-tam...,2024-02-28,2024-03-07,8 days
2,https://www.apolar.com.br/alugar/almirante-tam...,2024-02-28,2024-03-07,8 days
3,https://www.apolar.com.br/alugar/almirante-tam...,2024-02-28,2024-03-07,8 days
4,https://www.apolar.com.br/alugar/almirante-tam...,2024-02-28,2024-03-07,8 days
...,...,...,...,...
1628,https://www.apolar.com.br/alugar/sao-jose-dos-...,2024-02-28,2024-03-07,8 days
1629,https://www.apolar.com.br/alugar/sao-jose-dos-...,2024-02-28,2024-03-07,8 days
1630,https://www.apolar.com.br/alugar/sao-jose-dos-...,2024-02-28,2024-03-07,8 days
1631,https://www.apolar.com.br/alugar/sao-jose-dos-...,2024-02-28,2024-03-07,8 days


In [82]:
df.iloc[20547]['link']

'https://www.apolar.com.br/alugar/curitiba/portao/alugar-residencial-apartamento-curitiba-portao-106214'

In [75]:
df.loc[df['data_coleta'] == '2024-08-10'].sample(10)

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica,bairro,aluguel,condominio,iptu,seguro_incendio,area,banheiros,quartos,suites,vagas_garagem,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
20645,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/capa...,"Avenida Winston Churchill, 309, Capão Raso - C...","R$ 2.000,00, , Condomínio R$ 631,00, IPTU R$ 6...","95 m², 1 banheiro, 1 vaga, 2 quartos",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 95m², Área Tot...",Capao raso,2000.0,631.0,61.0,35.0,95,1,2,NaN,NaN,Não,Sim,Não,Sim,Sim,Sim
20694,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/sant...,"Rua Engenheiro Luiz Carlos De Oliveira Borges,...","R$ 1.700,00, , Condomínio R$ 390,00, IPTU R$ 5...","72 m², 1 vaga, 3 quartos",Apartamento com 3 qua...,"Cidade: Curitiba, Área Terreno: 72m², Área Tot...",Santa quiteria,1700.0,390.0,59.0,30.0,72,NaN,3,NaN,NaN,Não,Não,Não,Sim,Sim,Não
20541,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/bate...,"Alameda Dom Pedro Ii, 332, Batel - Curitiba","R$ 4.100,00, , Condomínio R$ 1.400,00, IPTU R$...","162 m², 2 banheiros, 1 vaga, 3 quartos, 1 suit...",Apartamento com 3 qua...,"Cidade: Curitiba, Área Terreno: 162.46m², Área...",Batel,4100.0,1400.0,197.0,53.0,162,2 s,3,1,NaN,Não,Não,Não,Não,Não,Sim
20547,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/port...,"Avenida República Argentina, 2751, Portão - Cu...","R$ 3.500,00, , Condomínio R$ 653,60, IPTU R$ 2...","90 m², 2 banheiros, 1 vaga, 2 quartos, 1 suite",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 90.11m², Área ...",Portao,3500.0,653.0,211.0,35.0,90,2 s,2,1,NaN,Sim,Não,Sim,Sim,Sim,Sim
20748,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/port...,"Rua João Bettega, 644, Portão - Curitiba","R$ 1.500,00, , Condomínio R$ 550,00, IPTU R$ 6...","57 m², 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 57m², Área Tot...",Portao,1500.0,550.0,61.0,27.0,57,NaN,2,NaN,NaN,Não,Não,Não,Sim,Não,Não
20734,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/cent...,"Rua Doutor Pedrosa, 134, Centro - Curitiba","R$ 1.550,00, , Condomínio R$ 780,00, IPTU R$ 9...","73 m², 1 banheiro, 2 quartos",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 73m², Área Tot...",Centro,1550.0,780.0,95.0,30.0,73,1,2,NaN,NaN,Sim,Não,Não,Não,Sim,Não
20819,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/jard...,"Rua Teófilo Soares Gomes, 892, Jardim Social -...","R$ 1.200,00, , Condomínio R$ 660,00, IPTU R$ 7...","91 m², 1 vaga, 1 quarto, pet",Apartamento com 1 qua...,"Cidade: Curitiba, Área Terreno: 91.44m², Área ...",Jardim social,1200.0,660.0,72.0,35.0,91,NaN,1 quarto,NaN,NaN,Não,Não,Não,Não,Não,Não
20858,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/siti...,"Rua Nova Aurora, 1570, Sítio Cercado - Curitiba","R$ 1.000,00, , Condomínio R$ 320,00, IPTU R$ 3...","51 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 51m², Área Tot...",Sitio cercado,1000.0,320.0,39.0,26.0,51,1,2,NaN,NaN,Não,Não,Não,Não,Não,Sim
20723,Apolar,2024-08-10,ref.,https://www.apolar.com.br/alugar/curitiba/cida...,", ,","R$ 0,00,",NaN,NaN,NaN,",",0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20557,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/bigo...,"Rua General Aristides Athayde Júnior, 602, Big...","R$ 3.000,00, , Condomínio R$ 965,61, IPTU R$ 6...","99 m², 1 banheiro, 1 vaga, 2 quartos",Apartament

In [146]:
import json

def request_chat_gpt(prompt):
    link = 'https://api.openai.com/v1/chat/completions'
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    body = {
        'model': id_modelo,
        'messages': [{'role':'user', 'content': prompt}]
    }

    res = requests.post(link, headers=headers, data= json.dumps(body))

    return res.json()['choices'][0]['message']['content']


def prepara_prompt(descricao):

    return f'''
    Você é um especialista em compra e venda de imóveis. Vou te passar a descrição de alguns anuncios de apartamentos e preciso que você organiza essas informações em um dicionário python sintetizando sempre algumas informações basicas:
    - Área (area)
    - Número de Quartos (quartos)
    - Número de Banheiros (banheiros)
    - Número de Suítes (suites)
    - Número de Vagas de Garagem (vagas_garagem)
    - Se o apartamento já possui alguma mobilia (camas, meses, comodas e etc) (mobilia)
    - Quais mobilias o apartamento possui (organize isso em uma lista dentro do dicionário) (lista_mobilias)
    - Se possui sacada e quantas possui (sacada)
    - Se o condomínio possui piscina (piscina)
    - Se o condomínio possui academia (academia)
    - Se o condomínio possui churrasqueira (churrasqueira)
    - Se o condomínio possui salão de festas (salao_de_festas)
    - Se o condomínio possui playground (playground)
    - Qual é o andar do apartamento (andar)
    - Se o aquecimento é a gás ou elétrico (aquecimento)
    - Se possui portaria (portaria)

    Se no texto não houver informações sobre os pontos solicitados, preencher o campo como "Não informado"
    Organize esses dados como um dicionário python.
    Reponda com apenas o dicionário
    O texto é o seguinte:

    {descricao}
    '''

In [147]:
samples = df.sample(5)

In [148]:
samples

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,ficha_tecnica,bairro,aluguel,condominio,iptu,seguro_incendio,area,banheiros,quartos,suites,vagas_garagem,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
17453,Apolar,2024-07-09,Apartamento Residencial para Locação no Centro...,https://www.apolar.com.br/alugar/curitiba/cent...,"Avenida Marechal Floriano Peixoto, 1900, Centr...","R$ 1.750,00, , Condomínio R$ 520,00, IPTU R$ 8...","93 m², 1 banheiro, 3 quartos, pet",Apartamento com 3 qua...,"Cidade: Curitiba, Área Terreno: 93.65m², Área ...",Centro,1750.0,520.0,82.0,35.0,93,1,3,NaN,NaN,Não,Não,Não,Não,Não,Não
20662,Apolar,2024-08-10,Apartamento Residencial para Locação no Bairro...,https://www.apolar.com.br/alugar/curitiba/cabr...,"Rua Cel Amazonas Marcondes, 981, Cabral - Curi...","R$ 1.900,00, , Condomínio R$ 400,00, IPTU R$ 8...","88 m², 1 vaga, 2 quartos",Apartamento com 2 qua...,"Cidade: Curitiba, Área Terreno: 88.14m², Área ...",Cabral,1900.0,400.0,80.0,34.0,88,NaN,2,NaN,NaN,Sim,Não,Não,Não,Não,Não
11458,Apolar,2024-05-07,Apartamento Residencial para Locação no Alto B...,https://www.apolar.com.br/alugar/curitiba/alto...,"Rua Capitão Amin Mosse, 101, Alto Boqueirão - ...","R$ 700,00, , IPTU R$ 6,86, Seguro Incêndio R$ ...","50 m², 1 banheiro, 1 quarto, pet",Apartamento com 1 qua...,"Cidade: Curitiba, Área Terreno: 50m², Área Tot...",Alto boqueirao,700.0,NaN,6.0,26.0,50,1,1 quarto,NaN,NaN,Não,Não,Sim,Não,Não,Não
9061,Apolar,2024-04-16,Apartamento Residencial para Locação no Novo M...,https://www.apolar.com.br/alugar/curitiba/novo...,"Rua Eduardo Carlos Pereira, 3989, Novo Mundo -...","R$ 1.800,00, , Condomínio R$ 391,88, IPTU R$ 4...","48 m², 1 banheiro, 1 vaga, 1 quarto",Apartamento com 1 qua...,"Cidade: Curitiba, Área Terreno: 48.89m², Área ...",Novo mundo,1800.0,391.0,47.0,26.0,48,1,1 quarto,NaN,NaN,Sim,Não,Não,Não,Não,Não
12542,Apolar,2024-05-18,Apartamento Residencial para Locação no Centro...,https://www.apolar.com.br/alugar/curitiba/cent...,"Rua Xv De Novembro, 1222, Centro - Curitiba","R$ 980,00, , Condomínio R$ 610,00, IPTU R$ 41,...","40 m², 1 banheiro, 1 quarto",Apartamento com 1 qua...,"Cidade: Curitiba, Área Terreno: 40m², Área Tot...",Centro,980.0,610.0,41.0,26.0,40,1,1 quarto,NaN,NaN,Não,Não,Não,Não,Não,Não


In [149]:
def coalesce(x):
    if pd.isna(x):
        return ''
    else:
        return x

In [150]:
samples['response_gpt'] = samples[['link','descricao', 'ficha_tecnica']].apply(lambda x: request_chat_gpt(prepara_prompt(x['descricao'] + '\n' + coalesce(x['ficha_tecnica']))), axis=1)

In [151]:
samples.loc[13087]['link']

KeyError: 13087

In [154]:
eval(samples['response_gpt'][17453])

{'area': '65 m²',
 'quartos': 3,
 'banheiros': 'Não informado',
 'suites': 'Não informado',
 'vagas_garagem': 'Não informado',
 'mobilia': 'Não',
 'lista_mobilias': [],
 'sacada': 'Não informado',
 'piscina': 'Não informado',
 'academia': 'Não informado',
 'churrasqueira': 'Não informado',
 'salao_de_festas': 'Não informado',
 'playground': 'Não informado',
 'andar': 'Térreo',
 'aquecimento': 'Não informado',
 'portaria': 'Não informado'}